### Importando bibliotecas:

In [10]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
import pickle

### Carregando dados:

In [11]:
# Carregando data frame de treino:

with open("train_df_final.pkl", "rb") as files:
    train_df = pickle.load(files)
train_df.head(5)

,comment_text,Toxic,pre-processado,pre-processado-sem-stopwords,quantidade_de_palavras,pre-processado-lemma,pre-processado-stem,pre-processado-sw-lemma,pre-processado-sw-stem
0,All of my edits are good. Cunts like you who ...,Toxic,all of my edits are good cunts like you who re...,edits good cunts like revert good edits stupid...,56,all of my edits are good cunt like you who rev...,all of my edit are good cunt like you who reve...,edits good cunt like revert good edits stupid ...,edit good cunt like revert good edit becaus st...
1,you are a stupid fuck \n\nand your mother's cu...,Toxic,you are a stupid fuck and your mother s cunt s...,stupid fuck mother cunt stinks,11,you are a stupid fuck and your mother s cunt s...,you are a stupid fuck and your mother s cunt s...,stupid fuck mother cunt stink,stupid fuck mother cunt stink
2,"""\n\nAnd you are? Let me know when you've craw...",Toxic,and you are let me know when you ve crawled o...,let know crawled arses dick laurent dead,18,and you are let me know when you ve crawled ou...,and you are let me know when you ve crawl out ...,let know crawled arses dick laurent dead,let know crawl ars dick laurent dead
3,Fuck off\n\nYou are NOT an administrator. You ...,Toxic,fuck off you are not an administrator you dono...,fuck administrator donot authority tell,18,fuck off you are not an administrator you dono...,fuck off you are not an administr you donot ha...,fuck administrator donot authority tell,fuck administr donot author tell
4,"Have you seen the reference to their claim, yo...",Toxic,have you seen the reference to their claim you...,seen reference claim moron source pov time red...,79,have you seen the reference to their claim you...,have you seen the refer to their claim you mor...,seen reference claim moron source pov time red...,seen refer claim moron thi sourc pov onc time ...


In [12]:
# Carregando dataframe de teste:

with open("test_df_final.pkl", "rb") as files:
    test_df = pickle.load(files)
test_df.head(3)

,comment_text,Toxic,pre-processado,pre-processado-sem-stopwords,quantidade_de_palavras,pre-processado-lemma,pre-processado-stem,pre-processado-sw-lemma,pre-processado-sw-stem
0,"== Arabs are committing genocide in Iraq, but ...",Toxic,arabs are committing genocide in iraq but no ...,arabs committing genocide iraq protests europe...,17,arab are committing genocide in iraq but no pr...,arab are commit genocid in iraq but no protest...,arab committing genocide iraq protest europe m...,arab commit genocid iraq protest europ may eur...
1,"I'd never think I'd need to say it, but Wikipe...",Non-Toxic,i d never think i d need to say it but wikiped...,never think need say wikipedia isnot fansite d...,33,i d never think i d need to say it but wikiped...,i d never think i d need to say it but wikiped...,never think need say wikipedia isnot fansite d...,never think need say wikipedia isnot fansit di...
2,"::No, he is an arrogant, self serving, immatur...",Toxic,no he is an arrogant self serving immature id...,arrogant self serving immature idiot get right,12,no he is an arrogant self serving immature idi...,no he is an arrog self serv immatur idiot get ...,arrogant self serving immature idiot get right,arrog self serv immatur idiot get right


In [13]:
# Modificando a coluna Toxic para binário:

train_df['Toxic'].replace('Non-Toxic', 0, inplace=True)
train_df['Toxic'].replace('Toxic', 1, inplace=True)

test_df['Toxic'].replace('Non-Toxic', 0, inplace=True)
test_df['Toxic'].replace('Toxic', 1, inplace=True)

### Iniciando a busca do melhor modelo:

In [14]:
# Lista com todas as colunas necessárias:

lista_de_colunas = {
    'comment_text',
    'pre-processado',
    'pre-processado-sem-stopwords',
    'pre-processado-lemma',
    'pre-processado-stem',
    'pre-processado-sw-lemma',
    'pre-processado-sw-stem'
}

In [15]:
# importando o config para utilizar os modelos e os vetorizadores:

from configs.config_models import *

In [17]:
dict_results = {}
df=pd.DataFrame()

for nome_vetorizador,vetorizador in vetorizadores.items():
    for nome_modelo, modelo in MODELOS.items():

        pipeline = Pipeline([
            ('vetorizador',vetorizador),
            ('modelo',modelo['clf'])
        ])

        params = {}
        for name_parameter, parameter in modelo['parameters'].items():
            params[f'modelo__{name_parameter}'] = parameter
            print('nome parametro: ', name_parameter)

        for coluna in lista_de_colunas:
            X_train = train_df[coluna]
            y_train = train_df['Toxic']

            X_test = test_df[coluna]
            y_test = test_df['Toxic']
                
            params.update(vetorizadores_params)

            randoms = RandomizedSearchCV(pipeline,param_distributions=params, cv=5, n_jobs=1, n_iter=5, verbose=2) 
            randoms.fit(X_train,y_train)

            with open(f'{coluna}_{nome_modelo}_{nome_vetorizador}.pickle', 'wb') as f:
                pickle.dump(randoms,f)

            y_pred = randoms.predict(X_test)
            acc = accuracy_score(y_test,y_pred)
            f1 = f1_score(y_test,y_pred)
            precision = precision_score(y_test,y_pred)
            matriz = confusion_matrix(y_test, y_pred)
            
            dict_results['nome_modelo']= (f'{coluna}_{nome_modelo}_{nome_vetorizador}')
            dict_results['acc'] = acc
            dict_results['f1'] = f1
            dict_results['precision'] = precision
            dict_results['matriz'] = matriz
            df = pd.concat([df, pd.DataFrame([dict_results])],ignore_index=True)
            print(f'Modelo {coluna}_{nome_modelo}_{nome_vetorizador}')

df        

nome parametro:  alpha
Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END modelo__alpha=0.55, vetorizador__max_df=0.6, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.55, vetorizador__max_df=0.6, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.55, vetorizador__max_df=0.6, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.55, vetorizador__max_df=0.6, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.55, vetorizador__max_df=0.6, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.3, vetorizador__max_df=0.7, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.3, vetorizador__max_df=0.7, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.3, vetorizador__max_df=0.7, vetorizador__ngram_range=(1, 1); total time=   0.0s
[CV] END modelo__alpha=0.3, vetorizador__max_df=0.7, vetorizador__ngram_

,nome_modelo,acc,f1,precision,matriz
0,comment_text_Bayes_tdidf,0.716000,0.824112,0.701828,"[[76, 424], [2, 998]]"
1,pre-processado-sw-lemma_Bayes_tdidf,0.698000,0.810540,0.696621,"[[78, 422], [31, 969]]"
2,pre-processado-stem_Bayes_tdidf,0.766000,0.850192,0.741623,"[[153, 347], [4, 996]]"
3,pre-processado-lemma_Bayes_tdidf,0.726000,0.829106,0.709609,"[[92, 408], [3, 997]]"
4,pre-processado-sem-stopwords_Bayes_tdidf,0.775333,0.854428,0.752091,"[[174, 326], [11, 989]]"
...,...,...,...,...,...
65,pre-processado-stem_SVM_bow,0.698667,0.814754,0.690278,"[[54, 446], [6, 994]]"
66,pre-processado-lemma_SVM_bow,0.807333,0.865893,0.807792,"[[278, 222], [67, 933]]"
67,pre-processado-sem-stopwords_SVM_bow,0.688667,0.809776,0.683162,"[[39, 461], [6, 994]]"
68,pre-processado_SVM_bow,0.808667,0.866945,0.808124,"[[278, 222], [65, 935]]"


### Verificando os melhores modelos:

In [18]:
df_sorted = df.sort_values(by = ['acc', 'precision'],ascending = False)
df_sorted[:5]

,nome_modelo,acc,f1,precision,matriz
36,pre-processado-sw-lemma_Bayes_bow,0.852000,0.894386,0.852995,"[[338, 162], [60, 940]]"
35,comment_text_Bayes_bow,0.848667,0.891750,0.852325,"[[338, 162], [65, 935]]"
38,pre-processado-lemma_Bayes_bow,0.848000,0.893358,0.839192,"[[317, 183], [45, 955]]"
40,pre-processado_Bayes_bow,0.846667,0.891509,0.843750,"[[325, 175], [55, 945]]"
37,pre-processado-stem_Bayes_bow,0.846000,0.890780,0.844843,"[[327, 173], [58, 942]]"


Podemos observar através dos 5 modelos considerados melhores, que Bayes se destacou. O vetorizador Bag of words obteve melhores resultados. Entre as colunas que se destacaram existiu uma variação de pre-processamentos, entretanto, cabe destacar que as colunas que passaram pelo processo de lemmatização se destacaram, pois apareceu 2 vezes no ranking.

Não podemos levar o f1-score muito em consideração pois o dataset é desbalanceado, devido à isso, as outras métricas tanto acutácia, como precisão e matriz de confusão, o melhor modelo é:    

### pre-processado-sw-lemma_Bayes_bow